# GreenPower RAG System v05 - Private Data Filter

## Pipeline complet:
1. Setup Qdrant (local vectoriel)
2. Chargement documents GreenPower via Gradio
3. Chunking + Embeddings (sentence-transformers)
4. Stockage Qdrant vector index avec **métadonnées de confidentialité**
5. RAG pipeline : query → retrieval → **filtrage private_** → Mistral
6. Interface Gradio interactive

## Nouveautés v05:
- ✨ **Système de confidentialité**: documents avec préfixe `private_` sont filtrés
- ✨ **Détection automatique**: lors de l'upload, détecte les fichiers privés
- ✨ **Message de confidentialité**: "Désolé, donnée confidentielle" pour les résultats privés
- ✨ **Logging transparent**: affiche quand des résultats confidentiels sont exclus

## Comment marquer un document comme privé:
- Nommez votre fichier avec le préfixe `private_` (ex: `private_salaires.pdf`)
- Le système détectera automatiquement et ajoutera un flag `is_private: true`
- Ces chunks ne seront jamais affichés dans les réponses

In [ ]:
# Cell 1: Installation des dépendances
!pip install -q langchain-mistralai langchain-community langchain-text-splitters
!pip install -q qdrant-client gradio sentence-transformers
!pip install -q pypdf python-docx python-dotenv

In [ ]:
# Cell 2: Imports
import os
from pathlib import Path
import gradio as gr
from langchain_mistralai import ChatMistralAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid
from typing import List
import pypdf
import docx
import json
import csv
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Imports OK")

In [ ]:
env_path = Path('.env')
if env_path.exists():
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
QDRANT_URL = os.getenv('QDRANT_URL', ':memory:')  # Use :memory: for local or cloud URL
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY', None)

In [ ]:
# Cell 4: Initialisation des clients

# Configuration CHUNKS
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize components
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
llm = ChatMistralAI(model='mistral-small-latest', mistral_api_key=MISTRAL_API_KEY, temperature=0.7)

qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
COLLECTION_NAME = "greenpower_docs"

# Text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

print("✅ Clients initialisés")
print(f"   - Qdrant: Mode {QDRANT_URL}")
print(f"   - Embeddings: {EMBEDDING_MODEL}")
print(f"   - LLM: Mistral Small")

In [ ]:
# Cell 5: Création de la collection Qdrant

# Obtenir la dimension des embeddings en générant un embedding de test
test_embedding = embeddings.embed_query("test")
VECTOR_SIZE = len(test_embedding)

def create_collection_if_not_exists():
    """Crée la collection Qdrant si elle n'existe pas déjà"""
    try:
        collections = qdrant_client.get_collections()
        collection_names = [c.name for c in collections.collections]
        
        if COLLECTION_NAME in collection_names:
            print(f"ℹ️ Collection '{COLLECTION_NAME}' existe déjà")
            return
        
        qdrant_client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=VECTOR_SIZE,
                distance=Distance.COSINE
            )
        )
        print(f"✅ Collection '{COLLECTION_NAME}' créée (dimension: {VECTOR_SIZE})")
    except Exception as e:
        print(f"⚠️ Erreur création collection: {e}")
        raise

# Créer la collection
create_collection_if_not_exists()

In [ ]:
# Cell 6: Fonctions utilitaires

def extract_text_from_file(file_path: str) -> str:
    """Extrait le texte d'un fichier PDF, DOCX, TXT, JSON ou CSV"""
    file_path = Path(file_path)
    
    try:
        if file_path.suffix.lower() == '.pdf':
            with open(file_path, 'rb') as f:
                pdf_reader = pypdf.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        
        elif file_path.suffix.lower() in ['.docx', '.doc']:
            doc = docx.Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        
        elif file_path.suffix.lower() == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        
        elif file_path.suffix.lower() == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Convertir le JSON en texte lisible
                if isinstance(data, dict):
                    text_parts = []
                    for key, value in data.items():
                        if isinstance(value, (dict, list)):
                            text_parts.append(f"{key}: {json.dumps(value, ensure_ascii=False, indent=2)}")
                        else:
                            text_parts.append(f"{key}: {value}")
                    return "\n".join(text_parts)
                elif isinstance(data, list):
                    return "\n\n".join([json.dumps(item, ensure_ascii=False, indent=2) for item in data])
                else:
                    return str(data)
        
        elif file_path.suffix.lower() == '.csv':
            with open(file_path, 'r', encoding='utf-8') as f:
                csv_reader = csv.DictReader(f)
                rows = []
                for row in csv_reader:
                    row_text = ", ".join([f"{key}: {value}" for key, value in row.items()])
                    rows.append(row_text)
                return "\n".join(rows)
        
        else:
            return f"Format non supporté: {file_path.suffix}"
    
    except Exception as e:
        return f"Erreur lecture fichier {file_path.name}: {str(e)}"

def is_private_file(filename: str) -> bool:
    """Détermine si un fichier est privé basé sur son nom"""
    return filename.lower().startswith('private_')

def chunk_and_embed(text: str, source: str, is_private: bool = False) -> List[dict]:
    """Découpe le texte en chunks et génère les embeddings avec flag de confidentialité"""
    # Chunking
    chunks = text_splitter.split_text(text)
    
    # Embeddings
    chunk_embeddings = embeddings.embed_documents(chunks)
    
    # Préparer les points pour Qdrant
    points = []
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        point = PointStruct(
            id=str(uuid.uuid4()),
            vector=embedding,
            payload={
                "text": chunk,
                "source": source,
                "chunk_index": i,
                "is_private": is_private  # 🔒 FLAG DE CONFIDENTIALITÉ
            }
        )
        points.append(point)
    
    return points

print("✅ Fonctions utilitaires définies (avec détection private_)")

In [ ]:
# Cell 7: Fonction d'upload de documents avec détection private_

def upload_documents(files):
    """Traite et stocke les documents uploadés avec détection de confidentialité"""
    if not files:
        return "❌ Aucun fichier uploadé"
    
    # S'assurer que la collection existe
    create_collection_if_not_exists()
    
    total_chunks = 0
    private_chunks = 0
    results = []
    
    for file in files:
        try:
            # Extraction du texte
            file_path = file.name if hasattr(file, 'name') else file
            filename = Path(file_path).name
            
            # 🔒 Détection si le fichier est privé
            is_private = is_private_file(filename)
            
            text = extract_text_from_file(file_path)
            
            if text.startswith("Erreur") or text.startswith("Format non supporté"):
                results.append(f"⚠️ {filename}: {text}")
                continue
            
            # Chunking et embedding avec flag private
            points = chunk_and_embed(text, filename, is_private=is_private)
            
            # Stockage dans Qdrant
            qdrant_client.upsert(
                collection_name=COLLECTION_NAME,
                points=points
            )
            
            total_chunks += len(points)
            if is_private:
                private_chunks += len(points)
                results.append(f"🔒 {filename}: {len(points)} chunks (PRIVÉ - ne sera pas affiché)")
            else:
                results.append(f"✅ {filename}: {len(points)} chunks (PUBLIC)")
            
        except Exception as e:
            results.append(f"❌ {filename}: Erreur - {str(e)}")
    
    summary = f"\n\n📊 **Total: {total_chunks} chunks** stockés dans '{COLLECTION_NAME}'\n"
    summary += f"   - 🔓 Public: {total_chunks - private_chunks} chunks\n"
    summary += f"   - 🔒 Privé: {private_chunks} chunks"
    
    return "\n".join(results) + summary

print("✅ Fonction upload_documents définie (avec marquage private_)")

In [ ]:
# Cell 8: Fonction RAG avec filtrage des données privées

def search_and_answer(question: str, top_k: int = 3):
    """Recherche dans Qdrant et génère une réponse avec Mistral (filtre les données privées)"""
    if not question or question.strip() == "":
        return "⚠️ Veuillez poser une question"
    
    try:
        # 1. Générer l'embedding de la question
        question_embedding = embeddings.embed_query(question)
        
        # 2. Recherche dans Qdrant avec un top_k plus large pour compenser le filtrage
        search_results = qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector=question_embedding,
            limit=top_k * 3  # Chercher 3x plus pour avoir assez après filtrage
        )
        
        # 3. 🔒 FILTRAGE DES DONNÉES PRIVÉES
        filtered_results = []
        private_count = 0
        
        for point in search_results:
            # Vérifier si le chunk est privé
            if point.payload.get("is_private", False):
                private_count += 1
                continue  # 🚫 Ignorer les chunks privés
            
            filtered_results.append(point)
            
            # Limiter au top_k demandé
            if len(filtered_results) >= top_k:
                break
        
        # 4. Vérifier s'il reste des résultats après filtrage
        if not filtered_results:
            if private_count > 0:
                return """## 🔒 Donnée confidentielle

Désolé, les informations pertinentes pour cette question sont marquées comme **confidentielles** et ne peuvent pas être affichées.

---

🔒 **{} résultats privés** ont été trouvés mais masqués pour des raisons de confidentialité.
""".format(private_count)
            else:
                return "⚠️ Aucun document trouvé. Uploadez d'abord des documents."
        
        # 5. Extraire les chunks pertinents (uniquement publics)
        contexts = []
        sources = []
        for point in filtered_results:
            contexts.append(point.payload["text"])
            source = point.payload.get("source", "Unknown")
            if source not in sources:
                sources.append(source)
        
        context_text = "\n\n---\n\n".join(contexts)
        
        # 6. Construire le prompt
        prompt = f"""Tu es un assistant qui répond aux questions en te basant UNIQUEMENT sur le contexte fourni.

Contexte:
{context_text}

Question: {question}

Instructions:
- Réponds de manière claire et concise
- Base-toi UNIQUEMENT sur le contexte fourni
- Si l'information n'est pas dans le contexte, dis-le clairement
- Cite les sources quand pertinent

Réponse:"""
        
        # 7. Générer la réponse avec Mistral
        response = llm.invoke(prompt)
        answer = response.content
        
        # 8. Formater la réponse avec info sur filtrage
        output = f"""## 💬 Réponse

{answer}

---

📚 **Sources consultées:** {', '.join(sources)}
🔍 **{len(contexts)} chunks** analysés (publics uniquement)
"""
        
        if private_count > 0:
            output += f"\n🔒 **{private_count} résultats privés** ont été exclus de cette recherche"
        
        return output
    
    except Exception as e:
        return f"❌ Erreur: {str(e)}\n\nDétails technique: {type(e).__name__}"

print("✅ Fonction search_and_answer définie (avec filtrage private_)")

In [ ]:
# Cell 9: Interface Gradio

with gr.Blocks(title="GreenPower RAG v05 - Private Filter") as demo:
    gr.Markdown(
        """
        # 🌱 GreenPower RAG System v05 - 🔒 Private Data Filter
        
        **Retrieval-Augmented Generation** pour vos documents GreenPower avec **gestion de confidentialité**
        
        ### 📋 Workflow:
        1. **Upload** vos documents (PDF, DOCX, TXT, JSON, CSV)
        2. Les fichiers avec préfixe `private_` seront marqués comme **confidentiels**
        3. **Ask** vos questions sur le contenu
        4. Les données privées ne seront **jamais affichées** dans les réponses
        
        ### 🔒 Confidentialité:
        - Fichiers commençant par `private_` → 🔒 **Masqués automatiquement**
        - Ex: `private_salaires.pdf`, `private_contrats.docx`
        """
    )
    
    with gr.Tab("📤 Upload Documents"):
        gr.Markdown(
            """
            ### Upload vos documents
            
            Formats supportés: **PDF**, **DOCX**, **TXT**, **JSON**, **CSV**
            
            🔒 **Documents privés:** Nommez vos fichiers avec le préfixe `private_`
            - `private_budget.pdf` → Sera marqué comme confidentiel
            - `rapport_public.pdf` → Sera accessible dans les réponses
            
            Les documents seront:
            - Découpés en chunks
            - Vectorisés avec sentence-transformers
            - Stockés dans Qdrant avec métadonnées de confidentialité
            """
        )
        
        file_input = gr.File(
            file_count="multiple",
            label="📁 Sélectionnez vos documents",
            file_types=[".pdf", ".docx", ".doc", ".txt", ".json", ".csv"]
        )
        upload_btn = gr.Button("🚀 Upload et Traiter", variant="primary")
        upload_output = gr.Textbox(
            label="📊 Résultat",
            lines=10,
            placeholder="Les résultats d'upload apparaîtront ici..."
        )
        
        upload_btn.click(
            upload_documents,
            inputs=file_input,
            outputs=upload_output
        )
    
    with gr.Tab("💬 Ask Questions"):
        gr.Markdown(
            """
            ### Posez vos questions
            
            Le système va:
            1. 🔍 Rechercher les passages pertinents
            2. 🔒 **Filtrer automatiquement les données privées**
            3. 🧠 Générer une réponse avec Mistral (uniquement données publiques)
            4. 📚 Citer les sources utilisées
            
            ⚠️ Si la réponse nécessite des données privées, vous verrez : **"Désolé, donnée confidentielle"**
            """
        )
        
        question_input = gr.Textbox(
            label="❓ Votre question",
            placeholder="Ex: Quels sont les objectifs de GreenPower pour 2025?",
            lines=3
        )
        
        top_k_slider = gr.Slider(
            minimum=1,
            maximum=10,
            value=3,
            step=1,
            label="🎯 Nombre de chunks à récupérer",
            info="Plus de chunks = plus de contexte (mais plus lent)"
        )
        
        ask_btn = gr.Button("🤔 Obtenir la Réponse", variant="primary")
        
        answer_output = gr.Markdown(
            label="💡 Réponse",
            value="*La réponse apparaîtra ici...*"
        )
        
        ask_btn.click(
            search_and_answer,
            inputs=[question_input, top_k_slider],
            outputs=answer_output
        )
        
        gr.Examples(
            examples=[
                ["Quels sont les objectifs principaux de l'entreprise?", 3],
                ["Quelle est la stratégie de développement?", 5],
                ["Quels sont les projets en cours?", 3],
            ],
            inputs=[question_input, top_k_slider],
        )
    
    with gr.Tab("ℹ️ Info"):
        gr.Markdown(
            f"""
            ### 🔧 Configuration Technique
            
            - **Vector DB:** Qdrant ({QDRANT_URL})
            - **Embeddings:** {EMBEDDING_MODEL}
            - **LLM:** Mistral Small
            - **Collection:** {COLLECTION_NAME}
            - **Chunk size:** {CHUNK_SIZE} caractères
            - **Overlap:** {CHUNK_OVERLAP} caractères
            
            ### 🔒 Système de Confidentialité
            
            **Comment ça marche:**
            1. Les fichiers avec préfixe `private_` sont détectés à l'upload
            2. Chaque chunk reçoit un flag `is_private: true/false` dans Qdrant
            3. Lors de la recherche, les chunks privés sont **automatiquement filtrés**
            4. Si tous les résultats sont privés → Message "Donnée confidentielle"
            
            **Exemple d'utilisation:**
            ```
            Documents:
            - rapport_2024.pdf → PUBLIC ✅
            - private_salaires.pdf → PRIVÉ 🔒
            - objectifs.docx → PUBLIC ✅
            - private_contrats_clients.csv → PRIVÉ 🔒
            
            Question: "Quels sont nos objectifs?"
            → Réponse basée uniquement sur rapport_2024.pdf et objectifs.docx
            
            Question: "Quels sont les salaires de l'équipe?"
            → "Désolé, donnée confidentielle" 🔒
            ```
            
            ### 📝 Notes
            
            - Mode in-memory ne persiste pas les données entre redémarrages
            - Pour la production, utilisez Qdrant en mode serveur ou cloud
            - Les embeddings sont générés localement (CPU)
            
            ### ✨ Nouveautés v05
            
            - ✅ Détection automatique des fichiers `private_*`
            - ✅ Flag `is_private` dans les métadonnées Qdrant
            - ✅ Filtrage intelligent lors de la recherche
            - ✅ Messages de confidentialité transparents
            - ✅ Compteur de résultats privés exclus
            """
        )

# Lancer l'interface
print("\n" + "="*60)
print("🚀 Lancement de l'interface Gradio v05 - Private Filter...")
print("="*60)

demo.launch(
    server_name="127.0.0.1",
    server_port=7651,
    share=False,
    show_error=True
)